In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.goodreads.com/quotes/tag/{}?page={}'

In [52]:
emotions = ['love', 'religion', 'humor','life', 'inspiration']

In [54]:
def get_quotes(complete):
    data = requests.get(complete)
    soup = BeautifulSoup(data.text)
    divs = soup.find_all('div', attrs={'class' : 'quoteText'})
    quotes = [div.text.strip().split('\n')[0][1:-1] for div in divs]
    return quotes

In [55]:
X, y = [], []

for emotion in emotions:
    for i in range(1, 6):
        complete = url.format(emotion, i)
        quotes = get_quotes(complete)
        X.extend(quotes)
        y.extend([emotion] * len(quotes))
        print(f'Processed page {i} for {emotion}')

Processed page 1 for love
Processed page 2 for love
Processed page 3 for love
Processed page 4 for love
Processed page 5 for love
Processed page 1 for religion
Processed page 2 for religion
Processed page 3 for religion
Processed page 4 for religion
Processed page 5 for religion
Processed page 1 for humor
Processed page 2 for humor
Processed page 3 for humor
Processed page 4 for humor
Processed page 5 for humor
Processed page 1 for life
Processed page 2 for life
Processed page 3 for life
Processed page 4 for life
Processed page 5 for life
Processed page 1 for inspiration
Processed page 2 for inspiration
Processed page 3 for inspiration
Processed page 4 for inspiration
Processed page 5 for inspiration


In [56]:
X[100]

'The very essence of romance is uncertainty.'

In [57]:
y[100]

'love'

In [58]:
import pandas as pd

In [59]:
df = pd.DataFrame(list(zip(y, X)), columns=['emotion', 'quotes'])

In [60]:
df.to_csv('emotions.csv', index=False)

In [61]:
from sklearn.feature_extraction.text import CountVectorizer

In [116]:
vect = CountVectorizer(max_features = 1000)

In [117]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [118]:
tokenizer = RegexpTokenizer('\w+')
sw = set(stopwords.words('english'))
ps = PorterStemmer()

In [119]:
def getStemmedQuote(quote):
    quote = quote.lower()
    
    # tokenize
    tokens = tokenizer.tokenize(quote)
    
    # remove stopwords
    new_tokens = [token for token in tokens if token not in sw]
    
    stemmed_token = [ps.stem(token) for token in new_tokens]
    
    clean_quote = ' '.join(stemmed_token)
    
    return clean_quote

def getStemmedQuotes(quotes):
    d = []
    for quote in quotes:
        d.append(getStemmedQuote(quote))
    return d

In [120]:
X = getStemmedQuotes(X)

In [121]:
vect.fit(X)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=1000, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [122]:
len(vect.vocabulary_)

1000

In [123]:
X_mod = vect.transform(X).todense()

In [124]:
from sklearn.model_selection import train_test_split

In [125]:
X_train, X_test, y_train, y_test = train_test_split(X_mod, y, test_size=0.33, random_state=42)

In [126]:
from sklearn.naive_bayes import BernoulliNB

In [127]:
model = BernoulliNB()

In [128]:
model.fit(X_train, y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [129]:
model.score(X_test, y_test)

0.42338709677419356

In [130]:
line = "You're just too good to be true can't take my eyes off you you'd be like heaven to touch I wanna hold you so much I love you baby"

In [131]:
X_vec = vect.transform([line]).todense()

In [132]:
model.predict(X_vec)

array(['love'], dtype='<U11')

In [133]:
model.score(X_train, y_train)

0.8207171314741036